In [ ]:
%matplotlib inline
class dummySessionState:
    """
    A simple class to mimic Streamlit's SessionState.
    Stores state as attributes and supports dict-like access.
    """
    def __init__(self):
        self._state = {}

    def __setitem__(self, key, value):
        self._state[key] = value

    def __getitem__(self, key):
        return self._state[key]

    def __contains__(self, key):
        return key in self._state

    def get(self, key, default=None):
        return self._state.get(key, default)

In [ ]:
import subprocess
import numpy as np
import platform_io
import pandas as pd
from copy import copy

# Import relevant libraries
import nidap_dashboard_lib as ndl   # Useful functions for dashboards connected to NIDAP
import basic_phenotyper_lib as bpl  # Useful functions for phenotyping collections of cells

def top_of_page_reqs(session_state):
    '''
    Top of the page requirements. These are the commands
    that are run at the top of the page that help maintain 
    '''

    # Initalize session_state values for streamlit processing
    session_state = ndl.init_session_state(session_state)

    # Check the platform
    session_state = check_for_platform(session_state)

    return session_state

def platform_is_nidap():
    '''
    Check if the Streamlit application is operating on NIDAP
    '''
    return np.any(['nidap.nih.gov' in x for x in subprocess.run('conda config --show channels', shell=True, capture_output=True).stdout.decode().split('\n')[1:-1]])

def check_for_platform(session_state):
    '''
    Set the platform parameters based on the platform the Streamlit app is running on
    '''
    # Initialize the platform object
    if not hasattr(session_state, 'platform'):
        session_state['platform'] = platform_io.Platform(platform=('nidap' if platform_is_nidap() else 'local'))
    return session_state

def apply_umap(session_state, umap_style):
    '''
    Call back function for applying the UMAP functions
    '''
    clust_minmax = [1, 40]
    session_state.bc.startTimer()
    session_state.spatial_umap = bpl.perform_spatialUMAP(spatial_umap = session_state.spatial_umap,
                                                         bc = session_state.bc,
                                                         UMAPStyle = umap_style)

    # Record time elapsed
    session_state.bc.printElapsedTime(msg = 'Performing UMAP')
    session_state.bc.set_value_df('time_to_run_UMAP', session_state.bc.elapsedTime())

    # List of possible UMAP Lineages as defined by the completed UMAP
    session_state.umapPheno = [session_state.def_lineage_opt]
    session_state.umapPheno.extend(session_state.pheno_summ['phenotype'])
    session_state.umapMarks = [session_state.def_lineage_opt]
    session_state.umapMarks.extend(session_state.spatial_umap.markers)
    session_state.umapMarks.extend(['Other'])

    # Identify all of the features in the dataframe
    session_state.outcomes = session_state.spatial_umap.cells.columns

    # List of possible outcome variables as defined by the config yaml files
    session_state.umapOutcomes = [session_state.def_umap_feature]
    session_state.umapOutcomes.extend(session_state.outcomes)
    session_state.inciOutcomes = [session_state.def_inci_feature]
    session_state.inciOutcomes.extend(session_state.outcomes)

    session_state.spatial_umap.prepare_df_umap_plotting(session_state.outcomes)

    session_state.df_umap = session_state.spatial_umap.df_umap

    # Perform possible cluster variations with the completed UMAP
    # session_state.bc.startTimer()
    # session_state.clust_range, session_state.wcss = bpl.measure_possible_clust(session_state.spatial_umap, clust_minmax)
    # session_state.bc.printElapsedTime(msg = 'Calculating possible clusters')

    session_state.wcss_calc_completed = True
    session_state.umapCompleted = True

    session_state = filter_and_plot(session_state)

    return session_state

def set_clusters(session_state):
    '''
    Callback function for setting the number of clusters
    and applying them to the UMAP/dataset
    '''

    session_state.bc.startTimer()
    session_state.spatial_umap = bpl.perform_clusteringUMAP(session_state.spatial_umap,
                                                            session_state.slider_clus_val,
                                                            session_state.clust_minmax,
                                                            session_state.cpu_pool_size)
    session_state.cluster_dict = session_state.spatial_umap.cluster_dict
    session_state.palette_dict = session_state.spatial_umap.palette_dict
    session_state.selected_nClus = session_state.slider_clus_val

    # Draw the 2D histogram UMAP colored by the clusters
    session_state.udp_full.cluster_dict = session_state.cluster_dict
    session_state.udp_full.palette_dict = session_state.palette_dict
    session_state.diff_clust_Fig = session_state.udp_full.umap_draw_clusters()

    # Record time elapsed
    session_state.bc.printElapsedTime(msg = 'Setting Clusters')
    session_state.bc.set_value_df('time_to_run_cluster', session_state.bc.elapsedTime())

    session_state.clustering_completed = True
    session_state = filter_and_plot(session_state)

    return session_state

def filter_and_plot(session_state):
    '''
    callback function to update the filtering and the 
    figure plotting
    '''

    session_state.prog_left_disabeled  = False
    session_state.prog_right_disabeled = False

    if session_state['idxSlide ID'] == 0:
        session_state.prog_left_disabeled = True

    if session_state['idxSlide ID'] == session_state['numSlide ID']-1:
        session_state.prog_right_disabeled = True

    if session_state.umapCompleted:
        session_state.spatial_umap.df_umap_filt = session_state.spatial_umap.df_umap.loc[session_state.spatial_umap.df_umap['Slide ID'] == session_state['selSlide ID'], :]
        session_state = ndl.setFigureObjs_UMAP(session_state, palette = st.session_state.palette_dict)

    return session_state

In [ ]:
files = ['input/pt_pt_pt_TnT 48h Untreated zScore areaNorm 48 h AA1 Reg_microns.csv',
         'input/ComboUntreatedNoH.csv',
         'input/Combo_CSVfiles_20230327_152849.csv',
         'input/mawa-unified_datafile-ScottLawrence_dataset-20240425_001550_EDT.csv']

columns_to_keep =[['tNt', 'GOODNUC', 'HYPOXIC', 'NORMOXIC', 'NucArea', 'RelOrientation'],
                  ['tNt', 'GOODNUC', 'HYPOXIC', 'NORMOXIC', 'NucArea', 'RelOrientation'],
                  ['Survival_5yr', 'Outcome'],
                  ['Survival_5yr', 'Outcome']]

markers_to_use = [['VIM', 'ECAD', 'NOS2', 'COX2'],
                  ['VIM', 'ECAD', 'NOS2', 'COX2'],
                  ['CD8', 'COX2', 'NOS2'],
                  ['CD68', 'CD8', 'CD3', 'PanCK']]

In [ ]:
# Import relevant libraries
import nidap_dashboard_lib as ndl   # Useful functions for dashboards connected to NIDAP
import basic_phenotyper_lib as bpl  # Useful functions for phenotyping collections of cells
import dataset_formats

session_state = dummySessionState()

# Run Top of Page (TOP) functions
session_state = top_of_page_reqs(session_state)

file_index = 2
selectProj_u = 'C:/DATA/neighborhood-profiles/'
filename = files[file_index]
dataset_path = selectProj_u + filename

extra_cols_to_keep = columns_to_keep[file_index]
marker_names = markers_to_use[file_index]

_, _, _, _, file_format, _ = dataset_formats.extract_datafile_metadata(filename)

dataset_class = getattr(dataset_formats, file_format)
dataset_obj = dataset_class(filename, 
                            coord_units_in_microns = 1, 
                            extra_cols_to_keep=extra_cols_to_keep)
dataset_obj.process_dataset()

# Load dataset into Memory
df_load = ndl.load_dataset(session_state.fiol, filename, files_dict = 'None', file_path = 'None', loadCompass=False)

# Perform pre-processing Steps
session_state = ndl.loadDataButton(session_state, dataset_obj.data, 'Local Upload', filename[:-4])

# Update 
session_state.marker_names = marker_names
session_state = ndl.set_phenotyping_elements(session_state, session_state.df_raw)

# Apply Phenotyping Method
session_state.selected_phenoMeth = 'Species'
session_state = ndl.updatePhenotyping(session_state)

In [ ]:
# Let us use the NeighborhoodProfiles class for performing spatial UMAP analysis
# This is something I would eventually like to integrate into the main workflow

from neighborhood_profiles import NeighborhoodProfiles, UMAPDensityProcessing

npf = NeighborhoodProfiles(bc = session_state.bc)

session_state.bc.startTimer()
npf.setup_spatial_umap(df = session_state.df,
                       marker_names = session_state.marker_multi_sel,
                       pheno_order  = session_state.phenoOrder,
                       smallest_image_size = session_state.datafile_min_img_size)

npf.perform_density_calc(calc_areas = False,
                         cpu_pool_size = session_state.cpu_pool_size,
                         area_threshold = 0.0)

npf.cell_counts_completed = True

In [ ]:
## Perform UMAP
session_state.bc.startTimer()
session_state = npf.perform_spatial_umap(session_state,
                                         umap_subset_per_fit= 20,
                                         umap_subset_toggle = False,
                                         umap_subset_per = 100)

# Record time elapsed
session_state.bc.printElapsedTime(msg = 'Performing UMAP')
session_state.bc.set_value_df('time_to_run_UMAP', session_state.bc.elapsedTime())

In [ ]:
# Perform clustering
npf.perform_clustering(5)

In [ ]:
import plotly.graph_objects as go

session_state.spatial_umap = npf.spatial_umap
df_umap = session_state.spatial_umap.df_umap
list_clusters = list(session_state.cluster_dict.values())
list_clusters.remove('No Cluster')
df_umap = ndl.filter_by_lineage(df_umap, session_state.lineageDisplayToggle_clus, session_state.def_lineage_opt, session_state.inciPhenoSel)

session_state.toggle_heatmap_filter_feat = True
session_state.heatmap_filter_feat = 'Survival_5yr'
session_state.heatmap_filter_value = 1

session_state.inciOutcomeSel = 'Survival_5yr'
displayas = 'Count Differences'

# Set up incidence dataframe
comp_thresh = None
inci_df = pd.DataFrame()
inci_df.index = list_clusters
inci_df['counts'] = 0
inci_df['featureCount1'] = 0 # True Condition
inci_df['featureCount0'] = 0 # False Condition

# Not Cell Counts
if session_state.inciOutcomeSel != session_state.def_inci_feature:
    col = df_umap[session_state.inciOutcomeSel]
    if ndl.identify_col_type(col) == 'not_bool':
        comp_thresh = 0
        df_umap['chosen_feature'] = df_umap.apply(lambda row: 1 if row[session_state.inciOutcomeSel] >= comp_thresh else 0, axis = 1)
    elif ndl.identify_col_type(col) == 'bool':
        df_umap['chosen_feature'] = df_umap[session_state.inciOutcomeSel]
    else:
        df_umap['chosen_feature'] = df_umap[session_state.inciOutcomeSel]

    # Compute the Difference
    for clust_label, group in df_umap.groupby('clust_label'):
        if clust_label != 'No Cluster':
            inci_df.loc[clust_label, 'counts'] = group['chosen_feature'].count()
            inci_df.loc[clust_label, 'featureCount1'] = sum(group['chosen_feature'] == 1)
            inci_df.loc[clust_label, 'featureCount0'] = sum(group['chosen_feature'] == 0)

    inci_df['Count Differences'] = inci_df['featureCount1'] - inci_df['featureCount0']

    sumf1 = sum(inci_df['featureCount1'])
    sumf0 = sum(inci_df['featureCount0'])

    inci_df['Percentages']  = 100*inci_df['featureCount1']/sumf1
    inci_df['Percentages0'] = 100*inci_df['featureCount0']/sumf0

    inci_df['Percentages1_adj'] = 100*(inci_df['featureCount1'] + 1)/(sumf1 + 1*session_state.selected_nClus)
    inci_df['Percentages0_adj'] = 100*(inci_df['featureCount0'] + 1)/(sumf0 + 1*session_state.selected_nClus)

    inci_df['Ratios'] = np.log10(inci_df['Percentages1_adj']/inci_df['Percentages0_adj'])
# Cell Counts
else:
    for clust_label, group in df_umap.groupby('clust_label'):
        if clust_label != 'No Cluster':
            inci_df.loc[clust_label, 'counts'] = group['Slide ID'].count()

feature = session_state.inciOutcomeSel
fig_title = 'Incidence by Cluster'
phenotype = session_state.inciPhenoSel
comp_thresh = comp_thresh
show_raw_counts = False

In [ ]:
inci_fig = go.Figure()

slc_bg   = '#0E1117'  # Streamlit Background Color
slc_text = '#FAFAFA'  # Streamlit Text Color
slc_bg2  = '#262730'  # Streamlit Secondary Background Color
slc_ylw  = '#F6EB61'  # Streamlit Yellow Color
slc_red  = '#FF4B4B'  # Streamlit Red Color


if comp_thresh is not None:
    up_tag = f' >= {comp_thresh}'
    dn_tag = f' < {comp_thresh}'
else:
    up_tag = ' = 1'
    dn_tag = ' = 0'

anno2 = False
if feature != 'Cell Counts':

    df = inci_df[displayas]

    dfmin = df.loc[(df != np.nan)].min()
    dfmax = df.loc[(df != np.nan)].max()
    up_limit = max(-1*dfmin, dfmax)
    if displayas == 'Count Differences':
        anno2 = True

        df_up = inci_df['featureCount1']
        df_dn = inci_df['featureCount0']

        dfmin = df_dn.loc[(df_dn != np.nan)].min()
        dfmax = df_up.loc[(df_up != np.nan)].max()
        up_limit = max(-1*dfmin, dfmax)

        if up_limit < 2:
            up_limit = 2
        ylim = [-1.05*up_limit, 1.05*up_limit]
        feature_pos = [1, up_limit*.95]
        feature_text = f'{feature}{up_tag}'
        feature_pos2 = [1, -up_limit*.95]
        feature_text2 =f'{feature}{dn_tag}'
        hover_template = '<b>Cluster:</b> %{x}<br><b>Count Difference:</b> %{y}<extra></extra>'
        outcome_suff = ' (Counts)'
    elif displayas == 'Ratios':
        anno2 = True
        ylim = [-1.05*up_limit, 1.05*up_limit]
        feature_pos = [1, up_limit*.95]
        feature_text = f'{feature}{up_tag}'
        feature_pos2 = [1, -up_limit*.95]
        feature_text2 =f'{feature}{dn_tag}'
        hover_template = '<b>Cluster:</b> %{x}<br><b>Ratio:</b> %{y}<extra></extra>'
        outcome_suff = ' Ratio (log10)'
    elif displayas == 'Percentages':
        ylim = [-1.05, 1.05*up_limit]
        feature_pos = [1, up_limit*.95]
        feature_text = f'{feature}{up_tag}'
        hover_template = '<b>Cluster:</b> %{x}<br><b>Percentage:</b> %{y}<extra></extra>'
        outcome_suff = ' (%)'
else:
    df = inci_df['counts']
    
    dfmin = df.min()
    dfmax = df.max()
    up_limit = max(-1*dfmin, dfmax)
    limrange = dfmax-dfmin
    liminc = limrange/8
    ylim = [0, dfmax + (liminc*0.1)]

    feature_pos = [1, up_limit*.95]
    feature_text = f'{feature}'
    hover_template = '<b>Cluster:</b> %{x}<br><b>Count:</b> %{y}<extra></extra>'
    outcome_suff = ' (Counts)'

if feature != 'Cell Counts':
    if displayas == 'Count Differences':
        if show_raw_counts:
            inci_fig.add_trace(go.Bar(
                x=df_up.index,
                y=df_up.values,
                name=f"{phenotype}{up_tag}",
                marker=dict(color=slc_ylw),
                hovertemplate=hover_template,
                hoverlabel=dict(
                bgcolor=slc_ylw,
                bordercolor=slc_ylw,
                font=dict(color=slc_bg)
                ),
                opacity=0.65,
                offsetgroup='1',
                showlegend=False,
                text=[f"<b>{int(y):,}</b>" for y in df_up.values],
                textposition='inside',
                textfont=dict(color=slc_bg, size=14)
            ))
            inci_fig.add_trace(go.Bar(
                x=df_dn.index,
                y=-df_dn.values,
                name=f"{phenotype}{dn_tag}",
                marker=dict(color=slc_red),
                hovertemplate=hover_template,
                hoverlabel=dict(
                bgcolor=slc_red,
                bordercolor=slc_red,
                font=dict(color=slc_text)
                ),
                opacity=0.65,
                offsetgroup='1',
                showlegend=False,
                text=[f"<b>{int(y):,}</b>" for y in df_dn.values],
                textposition='inside',
                textfont=dict(color=slc_bg, size=14)
            ))

    inci_fig.add_trace(go.Scatter(
        x=df.index,
        y=df.values,
        mode='lines+markers',
        name=phenotype,
        line=dict(color='#0E86D4', width=2.5),
        marker=dict(color='#0E86D4', size=14),
        hovertemplate=hover_template,
        hoverlabel=dict(
            bgcolor='#0E86D4',
            bordercolor='#0E86D4',
            font=dict(color=slc_text)
        )
    ))
else:
    inci_fig.add_trace(go.Bar(
        x=df.index,
        y=df.values,
        name=phenotype,
        marker=dict(color='#0E86D4'),
        hovertemplate=hover_template,
        hoverlabel=dict(
            bgcolor='#0E86D4',
            bordercolor='#0E86D4',
            font=dict(color=slc_text)
        )
    ))

annotations = [
    dict(
        x=feature_pos[0],
        y=feature_pos[1],
        text=feature_text,
        showarrow=False,
        font=dict(size=40, color=slc_text),
        xanchor='center',
        yanchor='bottom',
        opacity=0.3,
    )
]

# Add a second annotation if anno2 is True
if anno2:
    annotations.append(
        dict(
            x=feature_pos2[0],
            y=feature_pos2[1],
            text=feature_text2,
            showarrow=False,
            font=dict(size=40, color=slc_text),
            xanchor='center',
            yanchor='bottom',
            opacity=0.3,
        )
    )

inci_fig.update_layout(
    title=dict(
        text=fig_title,
        font=dict(size=25),
        x=0.08,  # Align title to the left (vertical axis)
        xanchor='left'
    ),
    xaxis_title="Cluster #",
    yaxis_title=f'{feature}{outcome_suff}',
    plot_bgcolor=slc_bg,
    paper_bgcolor=slc_bg,
    font=dict(color=slc_text, size=14),
    xaxis=dict(showgrid=True, gridcolor=slc_bg2, showline=False, zeroline=False),
    yaxis=dict(
        showgrid=True,
        gridcolor=slc_bg2,
        showline=False,
        zeroline=False,
        range=ylim
    ),
    width=2000,
    height=1000,
    legend=dict(
        title=None,
        bgcolor=slc_bg2,
        bordercolor=slc_bg2,
        borderwidth=1,
        orientation='v',
        x=0.85,  # Move legend outside the plot area to the right
        y=1,
        xanchor='left',
        yanchor='top'
    ),
    showlegend=True,
    annotations=annotations,
    shapes=[
        dict(
            type='line',
            xref='paper',
            x0=0,
            x1=1,
            yref='y',
            y0=0,
            y1=0,
            line=dict(
                color=slc_text,
                width=2,
                dash='dash'
            ),
            opacity=0.7,  # Lower opacity to push the reference line under the data
            layer='below'  # Ensures the line is rendered below traces
        )
    ]
)

inci_fig

In [ ]:
### Cluster/Phenotype Heatmap ###
if session_state.NormHeatRadio == 'Norm within Clusters':
    norm_axis = 0
elif session_state.NormHeatRadio == 'Norm within Phenotypes':
    norm_axis = 1
else:
    norm_axis = None

title = [f'DATASET: {session_state.datafile}',
         f'PHENO METHOD: {session_state.selected_phenoMeth}']

if session_state.toggle_heatmap_filter_feat:
    title.append(f'Filtered for data where {session_state.heatmap_filter_feat} = {session_state.heatmap_filter_value}')
    df_umap_filt = df_umap[df_umap[session_state.heatmap_filter_feat] == session_state.heatmap_filter_value]
else:
    df_umap_filt = df_umap

session_state.heatmapfig = bpl.draw_heatmap_fig(df_umap_filt,
                                                pheno_list=session_state.pheno_summ['phenotype'],
                                                title=title,
                                                norm_axis=norm_axis)

In [ ]:
session_state.spatial_umap.dens_df_mean

In [ ]:
session_state['sel_npf_fig'] = 'Cluster 1'
session_state['sel_npf_fig2'] = None
session_state['compare_clusters_as'] = 'Ratio'
session_state['toggle_hide_other'] = False
session_state['toggle_hide_no_cluster'] = False
session_state['nei_pro_toggle_log_scale'] = True

# Draw the Neighborhood Profile
npf_fig, ax = bpl.draw_scatter_fig(figsize=(14, 16))

bpl.draw_neigh_profile_fig(session_state.spatial_umap,
                           ax = ax,
                           sel_clus = session_state['sel_npf_fig'],
                           cmp_clus = session_state['sel_npf_fig2'],
                           cmp_style=session_state['compare_clusters_as'],
                           hide_other = session_state['toggle_hide_other'],
                           hide_no_cluster = session_state['toggle_hide_no_cluster'])

if session_state['nei_pro_toggle_log_scale']:
    ax.set_yscale('log')


In [ ]:
import seaborn as sns
import multiprocessing as mp
spatial_umap = npf.spatial_umap
n_clusters = 5
clust_minmax = session_state.clust_minmax
cpu_pool_size = session_state.cpu_pool_size

clust_range = range(clust_minmax[0], clust_minmax[1]+1)

kwargs_list = []
for clust in clust_range:
    kwargs_list.append(
        (
            spatial_umap.umap_test,
            clust
        )
    )

# Create a pool of worker processes
with mp.Pool(processes=cpu_pool_size) as pool:
    results = pool.starmap(bpl.kmeans_calc, kwargs_list)

wcss = [x.inertia_ for x in results]

# Create WCSS Elbow Plot
spatial_umap.elbow_fig = bpl.draw_wcss_elbow_plot(clust_range, wcss, n_clusters)

# Identify the kmeans obj that matches the selected cluster number
kmeans_obj_targ = results[n_clusters-1]

spatial_umap.cluster_dict = dict()
for i in range(n_clusters):
    spatial_umap.cluster_dict[i+1] = f'Cluster {i+1}'
spatial_umap.cluster_dict[0] = 'No Cluster'

spatial_umap.palette_dict = dict()
for i in range(n_clusters):
    spatial_umap.palette_dict[f'Cluster {i+1}'] = sns.color_palette('tab20')[i]
spatial_umap.palette_dict['No Cluster'] = 'white'

# Assign values to cluster_label column in df_umap
spatial_umap.df_umap.loc[:, 'clust_label'] = [spatial_umap.cluster_dict[key] for key in (kmeans_obj_targ.labels_+1)]

In [ ]:
session_state.slider_clus_val = 5
npf.spatial_umap = bpl.umap_clustering(npf.spatial_umap,
                                       session_state.slider_clus_val,
                                       session_state.clust_minmax,
                                       session_state.cpu_pool_size)

In [ ]:
### Perform Density Calculations

# Selection for Feature
session_state.toggle_clust_diff = True
session_state.dens_diff_feat_sel = 'HYPOXIC'
session_state.dens_diff_cutoff   = 0.05
session_state.num_clus_0 = 3
session_state.num_clus_1 = 3

# Create Full UMAP example
udp_full = UMAPDensityProcessing(npf, df = npf.spatial_umap.df_umap)
session_state.UMAPFig = udp_full.UMAPdraw_density()

In [ ]:

# Identify UMAP by Condition
session_state.df_umap_fals = npf.spatial_umap.df_umap.loc[npf.spatial_umap.df_umap[session_state.dens_diff_feat_sel] == 0, :]
session_state.df_umap_true = npf.spatial_umap.df_umap.loc[npf.spatial_umap.df_umap[session_state.dens_diff_feat_sel] == 1, :]

# Perform Density Calculations for each Condition
udp_fals = UMAPDensityProcessing(npf, session_state.df_umap_fals, xx=udp_full.xx, yy=udp_full.yy)
udp_true = UMAPDensityProcessing(npf, session_state.df_umap_true, xx=udp_full.xx, yy=udp_full.yy)

## Copy over
udp_diff = copy(udp_fals)
## Perform difference calculation
udp_diff.dens_mat = udp_true.dens_mat - udp_fals.dens_mat
## Rerun the min/max calcs
udp_diff.umap_summary_stats()
## Set Feature Labels
udp_fals.set_feature_label(session_state.dens_diff_feat_sel, '= 0')
udp_true.set_feature_label(session_state.dens_diff_feat_sel, '= 1')
udp_diff.set_feature_label(session_state.dens_diff_feat_sel, 'Difference')

# Draw UMAPS
session_state.UMAPFig_fals = udp_fals.UMAPdraw_density()
session_state.UMAPFig_true = udp_true.UMAPdraw_density()
session_state.UMAPFig_diff = udp_diff.UMAPdraw_density(diff= True)

# Assign Masking and plot
udp_mask = copy(udp_diff)
udp_mask.filter_density_matrix(session_state.dens_diff_cutoff)
udp_mask.set_feature_label(session_state.dens_diff_feat_sel, f'Difference- Masked, cutoff = {session_state.dens_diff_cutoff}')
session_state.UMAPFig_mask = udp_mask.UMAPdraw_density(diff= True)

# Perform Clustering
udp_clus = copy(udp_mask)
udp_clus.perform_clustering(dens_mat_cmp=udp_mask.dens_mat,
                            num_clus_0=session_state.num_clus_0, 
                            num_clus_1=session_state.num_clus_1)
udp_clus.set_feature_label(session_state.dens_diff_feat_sel, f'Clusters, False-{session_state.num_clus_0}, True-{session_state.num_clus_1}')
session_state.UMAPFig_clus = udp_clus.UMAPdraw_density(diff= True)
session_state.cluster_dict = udp_clus.cluster_dict

# if session_state['toggle_clust_diff']:

# Draw Visualizations
# 1. FULL UMAP
# 2. UMAP Separatated by Condition
# 3. UMAP Difference
# 4. UMAP Differences Masks
# 5. UMAP Differences Clusters

session_state.spatial_umap = npf.spatial_umap

In [ ]:
densUni = np.unique(udp_clus.dens_mat)
print(f'Density Shape = {np.shape(udp_clus.dens_mat)}', f'Unique values are {densUni}')
print(udp_full.bin_indices_df_group.shape)
udp_full.bin_indices_df_group

In [ ]:
# Add cluster label column to cells dataframe
session_state.spatial_umap.df_umap.loc[:, 'clust_label'] = 'No Cluster'
session_state.spatial_umap.df_umap.loc[:, 'cluster'] = 'No Cluster'
session_state.spatial_umap.df_umap.loc[:, 'Cluster'] = 'No Cluster'

for key, val in session_state.cluster_dict.items():
    bin_clust = np.argwhere(udp_clus.dens_mat == key)
    bin_clust = [tuple(x) for x in bin_clust]
    print(val, len(bin_clust))

    matching_indices = []
    # Iterate over each tuple in bin_clust
    for bin_tuple in bin_clust:
        # Get the X and Y values of the bin
        bin_x, bin_y = bin_tuple

        # Find the indices in udp_full.bin_indicies_df_group where X and Y match the bin's X and Y
        indices = udp_full.bin_indices_df_group[(udp_full.bin_indices_df_group['indx'] == bin_x) & (udp_full.bin_indices_df_group['indy'] == bin_y)].index

        # Append the indices to the list
        matching_indices.extend(indices)
    
    umap_ind = matching_indices
    # print(matching_indices)
    # print(val, len(umap_ind))
    session_state.spatial_umap.df_umap.loc[umap_ind, 'clust_label'] = val
    session_state.spatial_umap.df_umap.loc[umap_ind, 'cluster'] = val
    session_state.spatial_umap.df_umap.loc[umap_ind, 'Cluster'] = val

# After assigning cluster labels, perform mean calculations
session_state.spatial_umap.mean_measures()

# Create the Cluster Scatterplot
session_state = filter_and_plot(session_state)

In [ ]:
sorted(session_state.spatial_umap.df_umap['clust_label'].unique())


In [ ]:
npf_fig = bpl.draw_neigh_profile_fig(session_state.spatial_umap,
                               sel_clus= 'False_Cluster2',
                               cmp_clus = None)

In [ ]:
# Perform Clustering
session_state['toggle_clust_diff'] = False
session_state.spatial_umap = npf.spatial_umap
session_state.slider_clus_val = 5
session_state = set_clusters(session_state)

In [ ]:
npf_fig = bpl.draw_neigh_profile_fig(session_state.spatial_umap,
                               sel_clus=0,
                               cmp_clus=None)

## Plotting

In [ ]:
session_state = ndl.setFigureObjs(session_state)

In [ ]:
NeiProFig = bpl.draw_neigh_profile_fig(session_state.spatial_umap, 3)

In [ ]:
session_state = ndl.setFigureObjs_UMAP(session_state)

In [ ]:
session_state.UMAPFigType = 'Density'
session_state.lineageDisplayToggle == 'Phenotypes'

session_state.umapInspect_Ver = 'All Phenotypes'
session_state.umapInspect_Feat = 'NucArea'
session_state.diffUMAPSel_Ver = 'All Phenotypes'
session_state.diffUMAPSel_Feat = 'NucArea'

session_state.inciOutcomeSel = 'HYPOXIC'
session_state.inciPhenoSel = 'ECAD+'
session_state = ndl.setFigureObjs_UMAPDifferences(session_state)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

dfUMAP = pd.DataFrame(data = session_state.spatial_umap.umap_test, columns = ['X', 'Y'])
dfUMAP['Cluster'] = session_state.spatial_umap.cells['clust_label'].values[session_state.spatial_umap.cells['umap_test']]
dfUMAP['Lineage'] = session_state.spatial_umap.cells['Lineage'].values[session_state.spatial_umap.cells['umap_test']]
for outcome in session_state.outcomes:
    dfUMAP[outcome] = session_state.spatial_umap.cells[outcome].values[session_state.spatial_umap.cells['umap_test']]
clustOrder = sorted(dfUMAP['Cluster'].unique())

n_bins = 200
xx = np.linspace(np.min(dfUMAP['X']), np.max(dfUMAP['X']), n_bins + 1)
yy = np.linspace(np.min(dfUMAP['Y']), np.max(dfUMAP['Y']), n_bins + 1)
n_pad = 30

minXY = dfUMAP[['X', 'Y']].min()-1
maxXY = dfUMAP[['X', 'Y']].max()+1

dfUMAPI = dfUMAP.copy()
dfUMAPD = dfUMAP.copy()

# Lineage filtering
dfUMAPI = ndl.filter_by_lineage(session_state, dfUMAPI, session_state.def_lineage_opt, session_state.umapInspect_Ver)
dfUMAPD = ndl.filter_by_lineage(session_state, dfUMAPD, session_state.def_lineage_opt, session_state.diffUMAPSel_Ver)

# weights for inspection umap
if session_state.umapInspect_Feat != session_state.def_umap_feature:
    w_Ins = dfUMAPI[session_state.umapInspect_Feat]
    w_Ins, dfUMAPI = bpl.preprocess_weighted_umap(w_Ins, dfUMAPI)
else:
    w_Ins = None

# weights for difference umap
if session_state.diffUMAPSel_Feat != session_state.def_umap_feature:
    w = dfUMAPD[session_state.diffUMAPSel_Feat]
    w, dfUMAPD = bpl.preprocess_weighted_umap(w, dfUMAPD)
    w_DiffA = w
    w_DiffB = max(w) - w
    w_Diff  = w_DiffA - w_DiffB
else:
    w_DiffA = None
    w_DiffB = None
    w_Diff  = None


In [ ]:
from scipy import ndimage as ndi
gaussian_sigma=0.5
bins=200
w = w_Diff

X = dfUMAPD['X']
Y = dfUMAPD['Y']

b, _, _ = np.histogram2d(X, Y, bins=bins)
b = ndi.gaussian_filter(b.T, sigma=gaussian_sigma)

s, _, _ = np.histogram2d(X, Y, bins=bins, weights=w)
s = ndi.gaussian_filter(s.T, sigma=gaussian_sigma)

d = np.zeros_like(b)
# d[b > 0] = s[b > 0] / b[b > 0]
d = s
d = ndi.gaussian_filter(d, sigma=gaussian_sigma)

print(np.quantile(d[d > 0].flatten(), 0.97))
print(np.quantile(d[d < 0].flatten(), 0.03))

In [ ]:
if session_state.diffUMAPSel_Feat in session_state.outcomes_nBOOL:
    compThresh = 0
    w_DiffA = np.array(w > compThresh).astype('int')
    w_DiffB = max(w) - w
else:
    w_DiffA = w
    w_DiffB = max(w) - w

print(w_DiffA)

# Import Libraries
import sys
sys.path.append('C:\GitHub\SpatialUMAP')

# import plottingTools from Baras codebase
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import PlottingTools as umPT

# load spatial_umap object
data_dir = r'C:\DATA\neighborhood-profiles'
spatial_umap = pickle.load(open(data_dir + '/pkl/spatial_umap.pkl', 'rb'))

In [ ]:
# Define a color palette for processes going forward\n",
colorPalette = np.array([[131, 201, 255], [125, 239, 161], [109, 63, 192], [255, 171, 171],\
                         [0, 104, 201], [41, 176, 157], [213, 218, 229], [255, 43, 43],\
                         [232, 197, 77], [255, 209, 106], [131, 201, 255], [160, 120, 148], \
                         [178, 70, 181], [255, 135, 0], [0, 104, 0], [82, 166, 56],\
                         [166, 63, 152], [141, 166, 42], [130, 86, 133], [133, 54, 23],\
                         [9, 89, 133], [240, 135, 228], [240, 188, 176], [113, 32, 240],\
                         [57, 240, 223], [95, 166, 94], [94, 94, 65], [94, 51, 51],\
                         [50, 94, 32], [252, 226, 17]])/256\
# umPT.draw_cmp_swatches(colorPalette)"

In [ ]:
# Define some labels for our phenotypes
phenoLabel = np.sort(spatial_umap.cells['Lineage'].unique())[::-1]
phenoColor = colorPalette

# Create a dictionary of phenotypes and the colors to draw them as
phenoSet = dict([(phenoLabel[x], phenoColor[x]) for x in range(len(phenoLabel))])

In [ ]:
import matplotlib as mpl
phenoLabel = np.sort(spatial_umap.cells['Lineage'].unique())[::-1]
phenoColor = mpl.colormaps['tab20'].colors

# Create a dictionary of phenotypes and the colors to draw them as
spatial_umap.pheno_palette_dict = dict([(phenoLabel[x], phenoColor[x]) for x in range(len(phenoLabel))])

In [ ]:
import pandas as pd
pd.__version__

### Cellular Spatial Positions
One of the first steps that will be useful is to have a view of the full cellular plate. Phenotypes can eventually be added, but for now, lets get a bird eye's view of the sample to confirm the shape of the cells.

In [ ]:
SpatPosFig = plt.figure(figsize = (7,7))
ax = SpatPosFig.add_subplot(1,1,1)
umPT.plot_spatial_elem(ax, 
                  elems = spatial_umap.cells, 
                  title=f'Spatial Cell Positions', 
                  color= spatial_umap.cells['Lineage'].map(phenoSet))

SpatPosFig.savefig(data_dir + '/plots/SampleSpatialPos.png')

In [ ]:
# umPT.plot_spatial_interactive(elems = spatial_umap.cells, title=f'Spatial Cell Positions', feature= 'Lineage')

### Neighborhood Profiles
The core of this analysis is to show varying densities of the cell phenotypes surrounding any given cell. For this reason, we need a fairly robust method of drawing this feature repeatability and comparatively.

In [ ]:
NeiProFig = plt.figure(facecolor = SlBgC, figsize = (20, 15))
NeiProFig.suptitle('Cell Densities', fontsize=55, color = SlTC)

maxDens = spatial_umap.density.max().max() # This will be a good var to use when outliers are corrected for

# Look at four (4) representative cells to get an idea of the neighborhood profiles.
for count, j in enumerate([1, 24, 2456, 4500]):
    density = spatial_umap.density[j, :, :]

    ax = NeiProFig.add_subplot(2, 2, count+1)
    if count+1 == 3:
        legF = 1
    else:
        legF = 0

    lineage = spatial_umap.cells.loc[j, "Lineage"]
    umPT.plot_neighborhood_profile(ax, f'{j}\n {lineage}', spatial_umap.dist_bin_um, density, phenoSet, 0.1, legF)
NeiProFig.savefig(data_dir + '/plots/SampleNeighborhoodProfiles.png')

In [ ]:
NeiPro_Proportion = plt.figure(facecolor = SlBgC, figsize = (20, 15))
NeiPro_Proportion.suptitle('Cell Proportions', fontsize=55, color = SlTC)

# Look at four (4) representative cells to get an idea of the neighborhood profiles.
for count, j in enumerate([1, 24, 2456, 4500]):
    propor = spatial_umap.proportion[j, :, :]

    ax = NeiPro_Proportion.add_subplot(2, 2, count+1)
    if count+1 == 4:
        legF = 1
    else:
        legF = 0

    lineage = spatial_umap.cells.loc[j, "Lineage"]
    umPT.plot_neighborhood_profile_propor(ax, f'{j}\n {lineage}', spatial_umap.dist_bin_um, propor, phenoSet, colorPalette, legF=legF)
NeiPro_Proportion.savefig(data_dir + '/plots/SampleNeighborhoodProfiles_Proportions.png')

### Spatial UMAP
Use a 2D histogram to display the heatmap of the 2-dimenstional UMAP features

In [ ]:
# Spatial UMAP 2D Density Plots By Lineage and with PD-L1 and PD1 overlays
# https://matplotlib.org/stable/tutorials/colors/colormaps.html

# set meshgrid / bins for 2d plots based on UMAP x, y distributions
n_bins = 200
xx = np.linspace(np.min(spatial_umap.umap_test[:, 0]), np.max(spatial_umap.umap_test[:, 0]), n_bins + 1)
yy = np.linspace(np.min(spatial_umap.umap_test[:, 1]), np.max(spatial_umap.umap_test[:, 1]), n_bins + 1)
n_pad = 30

# get figure and axes
sUMAPFig, ax = plt.subplots(4, 5, figsize=(24, 20), facecolor='white')

# color maps
cmap_dens = plt.get_cmap('viridis').copy()
cmap_dens.set_under('white')
cmap_div = plt.get_cmap('coolwarm').copy()

# plot cmaps
umPT.plt_cmap(ax=ax[1, 4], cmap=cmap_dens, extend='max', width=0.01, ylabel='Density')
umPT.plt_cmap(ax=ax[2, 4], cmap=cmap_div, extend='both', width=0.01, ylabel='Rel. Nucleus Ori')
umPT.plt_cmap(ax=ax[3, 4], cmap=cmap_div, extend='both', width=0.01, ylabel='Nucleus Area')

# clear unneeded axes
[ax[_].set(visible=False) for _ in [(0, 0), (0, 1), (0, 3), (0, 4)]]

# plot 2d denisty in umap of all cells
umPT.plot_2d_density(spatial_umap.umap_test[:, 0], spatial_umap.umap_test[:, 1], bins=[xx, yy], n_pad=n_pad, ax=ax[0, 2], cmap=cmap_dens)
ax[0, 2].set(title='All Cells')

# set lineages to show and in what order
lineages_plot = ['VIM', 'ECAD', 'COX2', 'NOS2']

# Secondary Characteristics-Rel Orientation
w = {'relOrientation': spatial_umap.cells['RelOrientation'].values}

# Secondary Characteristics-Nuc Area
w['NucArea'] = spatial_umap.cells['NucArea'].values

for i in range(len(lineages_plot)):
    # cells of lineage(s)
    idx = spatial_umap.cells['Lineage'].str.contains(f'{lineages_plot[i]}\+')
    ax[1, i].cla()
    ax[1, i].set(title=lineages_plot[i])

    # plot density
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], ax=ax[1, i], cmap=cmap_dens, vlim=.95)

    # plot Secondary Pheno 1
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], w=w['relOrientation'][idx], ax=ax[2, i],
                    cmap=cmap_div, vlim=np.array([spatial_umap.cells['RelOrientation'].min(), spatial_umap.cells['RelOrientation'].max()]), circle_type='arch')
    # plot Secondary Pheno 2
    umPT.plot_2d_density(spatial_umap.umap_test[idx, 0], spatial_umap.umap_test[idx, 1], bins=[xx, yy], w=w['NucArea'][idx], ax=ax[3, i], 
                    cmap=cmap_div, vlim=np.array([spatial_umap.cells['NucArea'].min(), spatial_umap.cells['NucArea'].max()]), circle_type='arch')

sUMAPFig.savefig(data_dir + '/plots/Sample-SpatialUMAP.png')

### Spatial Coord/UMAP

In [ ]:
clust_set = np.sort(spatial_umap.cells['clust_label'].unique())
color_set = dict([(clust_set[x], colorPalette[x]) for x in range(len(clust_set))])

SpatPosClustFig = plt.figure(figsize = (7,7))
ax = SpatPosClustFig.add_subplot(1,1,1)
umPT.plot_spatial_elem(ax, 
                  elems = spatial_umap.cells, 
                  title=f'Spatial Cell Positions\nWith Spatial-UMAP cluster Overlay', 
                  color=spatial_umap.cells['clust_label'].map(color_set))

SpatPosClustFig.savefig(data_dir + '/plots/SampleSpatialPosUMAP.png')

In [ ]:
# umPT.plot_spatial_interactive(elems = spatial_umap.cells, title=f'Spatial Cell Positions-UMAP Overlay', feature= 'clust_label')

### Meaned Measures (Density/Proportion)

In [ ]:
# Max density value across all densities
maxDens = spatial_umap.density.max().max() # This will be a good var to use when outliers are corrected for

# Create figure and axes
NeiProFigMean = plt.figure(figsize=(12,12), facecolor = SlBgC)
ax = NeiProFigMean.add_subplot(1, 1, 1, facecolor = SlBgC)

umPT.plot_mean_neighborhood_profile(ax, 10, spatial_umap.dist_bin_um, spatial_umap.densMeansDict, spatial_umap.pheno_palette_dict, 0.1, legF=1)